# SQL Databases
- easy to use from Python
- many different databases are supported: 
    - sqlite3
    - mysql
    - postgres
    - oracle
- also NoSql

In [8]:
# need some names to build a database, 
# so made some random strings
# but they are hard to read...

import random
import string

string.ascii_lowercase

def makeNames(n):
    # chr takes an ascii code and 
    # returns the letter in a string
    n = ''.join([random.choice(string.ascii_lowercase) \
                 for j in range(n)])
    return n.capitalize()
    
[makeNames(4) for j in range(10)]

['Sxbm',
 'Uqmy',
 'Joai',
 'Dcjc',
 'Rnfq',
 'Merb',
 'Loyu',
 'Pgjr',
 'Gera',
 'Ooct']

In [2]:
# nltk corpus has 8,000 real names!

import random
import nltk

def makeNames(wcnt, wlen):
    names = [w for w in nltk.corpus.names.words() \
             if len(w) == wlen]
    # names are in alphabetical order
    # random doesn't have a 'random subset' routine
    # so do a shuffle 
    random.shuffle(names)
    # then just take random names from the front
    return names[:wcnt]

# 4 letter first names
firsts = makeNames(10, 4)
# 5 letter last names
lasts = makeNames(10, 5)

[firsts, lasts]

LookupError: 
**********************************************************************
  Resource [93mnames[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('names')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/names[0m

  Searched in:
    - '/Users/dbenson30/nltk_data'
    - '/Users/dbenson30/opt/anaconda3/nltk_data'
    - '/Users/dbenson30/opt/anaconda3/share/nltk_data'
    - '/Users/dbenson30/opt/anaconda3/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


# [sqlite3](https://www.sqlite.org/index.html)
- sqlite3 stores the entire database in one file
    - very easy to use, no config
- world's most deployed sql db(used in cellphones)
- [driver doc](https://docs.python.org/3.7/library/sqlite3.html)

In [1]:
# make a db connection and get a cursor

import sqlite3 
from pathlib import Path
import tempfile
path = Path(tempfile.NamedTemporaryFile().name)

con = sqlite3.connect(path)
cur = con.cursor()
path, cur

(PosixPath('/var/folders/2z/vj69b89s1xxfb51stm_z4fnr0000gr/T/tmp70vqqovz'),
 <sqlite3.Cursor at 0x1119bd3b0>)

# Create two tables
- grade(uni, course, grade)
- student(first, last, uni)

In [2]:
cur.execute('create table grade (uni text, course text, grade real)')

In [3]:
cur.execute('create table student (first text, last text, uni text)')

# Build student table

In [6]:
# first, last

firsts = ["david", "bob"]
lasts = ["benson", "bob2"]

fl = [ [firsts[k], lasts[k]] for k in range(2)]
fl

[['david', 'benson'], ['bob', 'bob2']]

In [9]:
# make an uni
# student = [first, last, uni]

students = [ [f, l, (f[0] + l[0] + str(random.randint(1000,9999))).lower()] 
            for f,l in fl]
students

[['david', 'benson', 'db4222'], ['bob', 'bob2', 'bb5179']]

In [10]:
# insert above list into db

# 'execute' inserts one row at a time
# expects one row of data
cur.execute('insert into student values(?, ?, ?)', students[0])

# 'executemany' inserts multiple rows at once
# expects list of rows, executes the statement for each row
cur.executemany('insert into student values(?, ?, ?)', \
                students[1:])

'''
# the above two statements are equivalent to 
for student in students:
    cur.execute('insert into student values(?, ?, ?)', student)

# or 
cur.executemany('insert into student values(?, ?, ?)', students)

executemany is typically more efficient 
'''

None

In [11]:
# the select returns a generator

sg = cur.execute('select first, last, uni from student')
sg

In [12]:
rows = list(sg)
rows

[('david', 'benson', 'db4222'), ('bob', 'bob2', 'bb5179')]

# Build grade table

In [13]:
# pull the uni out of the tuple 

unis = [t[0] for t in cur.execute('select uni from student')]
unis

['db4222', 'bb5179']

In [14]:
# classes 

classes = ['COMSW {}'.format(random.randint(1000, 9999)) \
           for j in range(5)]
classes

['COMSW 9018', 'COMSW 6564', 'COMSW 2856', 'COMSW 6449', 'COMSW 4969']

In [21]:
# students take finals

grades = [ [u , random.choice(classes), \
            70 + 30 * random.random() ] \
          for u in unis ]
grades

[['db4222', 'COMSW 4969', 81.53564925234112],
 ['bb5179', 'COMSW 9018', 84.4582229609556]]

In [22]:
# executemany inserts multiple rows at once 

cur.executemany('insert into grade values(?, ?, ?)', grades)

In [23]:
# join the two tables on the uni field

list(cur.execute('select first,last,student.uni, \
                 grade from student,grade \
                 where student.uni = grade.uni'))

[('david', 'benson', 'db4222', 73.55910043406739),
 ('david', 'benson', 'db4222', 81.53564925234112),
 ('bob', 'bob2', 'bb5179', 72.5412096799984),
 ('bob', 'bob2', 'bb5179', 84.4582229609556)]

In [24]:
# add a filter term

q = 'select last,first,grade from student,grade \
 where grade>80 and student.uni = grade.uni \
 order by last'
list(cur.execute(q))

[('benson', 'david', 81.53564925234112), ('bob2', 'bob', 84.4582229609556)]

In [25]:
# always commit and close the connection

con.commit()
con.close()

In [29]:
# data is persisted on disk, can read again
# use 'with' to manage db connection

res=None
res2=None

with sqlite3.connect(path) as con:
    cur = con.cursor()
    res = list(cur.execute("select name from sqlite_master where type = 'table';"))
    res2 = list(cur.execute(q))


res, res2

([('grade',), ('student',)],
 [('benson', 'david', 81.53564925234112), ('bob2', 'bob', 84.4582229609556)])

In [27]:
spath = str(path)
spath

'/var/folders/2z/vj69b89s1xxfb51stm_z4fnr0000gr/T/tmp70vqqovz'

In [28]:
# want to use this in orm notebook
# there is a bug - path object gets messed up, 
# so pass string

%store spath

Stored 'spath' (str)


# NoSQL databases
- provide less functionality than SQL, but are more efficient and scale better
- [mongodb](https://www.mongodb.org) is a popular one
- [PyMongo](https://docs.mongodb.org/getting-started/python/client/) is the python driver for mongodb
